In [32]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import r2_score
from sklearn import datasets as dss
from sklearn.model_selection import train_test_split
import warnings
import math
warnings.filterwarnings('ignore')

In [33]:
class housing_ds(torch.utils.data.Dataset):

    def __init__(self, init_dataset, init_target, transform=None):
        self._base_dataset = torch.from_numpy(init_dataset).type(torch.float)
        self._base_targets = torch.from_numpy(init_target).type(torch.float)
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        features = self._base_dataset[idx]
        target = self._base_targets[idx]

        if self.transform is not None:
            features = self.transform(features)

        return features, target

In [34]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation

    def forward(self, x):
        x = self.fc(x)
        if self.activation == "relu":
            return F.relu(x)
        if self.activation == "sigmoid":
            return F.sigmoid(x)
        if self.activation == "leaky_relu":
            return F.leaky_relu(x)
        raise RuntimeError


class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.fc1 = Perceptron(input_dim, hidden_dim, 'leaky_relu')
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.15)
        self.fc2 = Perceptron(hidden_dim, 1, 'leaky_relu')

    def forward(self, x):
        x = self.fc1(x)
        x = self.dp(x)
        x = self.bn(x)
        x = self.fc2(x)

        return x

In [35]:
data = dss.fetch_california_housing()

In [36]:
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size = 0.25, random_state = 13)

In [37]:
train_dataset = housing_ds(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=150, shuffle=False)

In [38]:
test_dataset = housing_ds(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False)

In [39]:
net = FeedForward(8, 8)

# optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
# optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.001)

criterion = nn.MSELoss()

In [40]:
if torch.cuda.is_available():
    net.cuda()

In [41]:
num_epochs = 10

for epoch in range(num_epochs):
    running_loss, running_items, r2 = 0.0, 0.0, 0.0

    for i, data in enumerate(train_loader):
        fets, target = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(fets)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(target)

        predict = outputs.data.numpy()
        tr_target = target.view(target.shape[0], 1).numpy()
        r2 += r2_score(tr_target, predict)

        # выводим статистику о процессе обучения
        if i % 30 == 0:
            net.eval()

            data = list(test_loader)[0]

            test_outputs = net(data[0])
            test_predict = test_outputs.data.numpy()
            te_target = data[1].view(data[1].shape[0], 1)
            test_r2 = r2_score(te_target, test_predict)

            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'r2: {r2:.3f}. ' \
                  f'Test r2: {test_r2:.3f}')

            running_loss, running_items, r2 = 0.0, 0.0, 0.0

            net.train()

print('Training is finished!')

Epoch [1/10]. Step [1/104]. Loss: 0.040. r2: -3.102. Test r2: -4.825
Epoch [1/10]. Step [31/104]. Loss: 0.033. r2: -86.344. Test r2: -3.527
Epoch [1/10]. Step [61/104]. Loss: 0.030. r2: -74.137. Test r2: -3.881
Epoch [1/10]. Step [91/104]. Loss: 0.026. r2: -62.145. Test r2: -2.297
Epoch [2/10]. Step [1/104]. Loss: 0.025. r2: -1.573. Test r2: -1.745
Epoch [2/10]. Step [31/104]. Loss: 0.020. r2: -41.227. Test r2: -2.327
Epoch [2/10]. Step [61/104]. Loss: 0.018. r2: -31.272. Test r2: -0.399
Epoch [2/10]. Step [91/104]. Loss: 0.016. r2: -25.368. Test r2: -0.115
Epoch [3/10]. Step [1/104]. Loss: 0.017. r2: -0.841. Test r2: -1.272
Epoch [3/10]. Step [31/104]. Loss: 0.014. r2: -21.084. Test r2: -0.141
Epoch [3/10]. Step [61/104]. Loss: 0.014. r2: -17.391. Test r2: 0.098
Epoch [3/10]. Step [91/104]. Loss: 0.012. r2: -13.420. Test r2: -0.012
Epoch [4/10]. Step [1/104]. Loss: 0.013. r2: -0.277. Test r2: -0.031
Epoch [4/10]. Step [31/104]. Loss: 0.011. r2: -9.394. Test r2: -0.170
Epoch [4/10]. St

C помощью оптимизатора Адам модель быстрее находит решение, в целом модель не очень хорошо справляется с задачей регрессии.